# 【問題1】クロスバリデーション
* train_test_splitではなく、クロスバリデーションを用いる
* クロスバリデーションには、scikit-learnのKFold ライブラリを使用する  

In [21]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [3, 4], [3, 4], [3, 4], [3, 4], [3, 4], [3, 4]])
y = np.array([0, 1, 2, 3, 4, 1, 1, 1, 1, 1])

# 5分割
skf = StratifiedKFold(n_splits=5)

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print("X_train:", X_train, "\nX_test:", X_test, "\ny_train:", y_train, "\ny_test:", y_test)

X_train: [[3 4]
 [3 4]
 [3 4]
 [3 4]] 
X_test: [[1 2]
 [3 4]
 [1 2]
 [3 4]
 [3 4]
 [3 4]] 
y_train: [1 1 1 1] 
y_test: [0 1 2 3 4 1]
X_train: [[1 2]
 [3 4]
 [1 2]
 [3 4]
 [3 4]
 [3 4]
 [3 4]
 [3 4]
 [3 4]] 
X_test: [[3 4]] 
y_train: [0 1 2 3 4 1 1 1 1] 
y_test: [1]
X_train: [[1 2]
 [3 4]
 [1 2]
 [3 4]
 [3 4]
 [3 4]
 [3 4]
 [3 4]
 [3 4]] 
X_test: [[3 4]] 
y_train: [0 1 2 3 4 1 1 1 1] 
y_test: [1]
X_train: [[1 2]
 [3 4]
 [1 2]
 [3 4]
 [3 4]
 [3 4]
 [3 4]
 [3 4]
 [3 4]] 
X_test: [[3 4]] 
y_train: [0 1 2 3 4 1 1 1 1] 
y_test: [1]
X_train: [[1 2]
 [3 4]
 [1 2]
 [3 4]
 [3 4]
 [3 4]
 [3 4]
 [3 4]
 [3 4]] 
X_test: [[3 4]] 
y_train: [0 1 2 3 4 1 1 1 1] 
y_test: [1]


/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


* Home Credit Default Risk コンペティションのデータセットでやってみる
* stratifyする

In [20]:
import pandas as pd

df_train = pd.read_csv('application_train.csv')

# 説明変数
X = df_train.drop(['SK_ID_CURR', 'TARGET'], axis = 1)

# 目的変数
y = df_train['TARGET']

# 5分割
skf = StratifiedKFold(n_splits=5)

for train_index, test_index in skf.split(X, y):
    print(train_index)
    print(test_index)
    # すげー出てくるのでコメントアウト
    #X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    #y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    #print("X_train:", X_train, "\nX_test:", X_test, "\ny_train:", y_train, "\ny_test:", y_test)

[ 61465  61466  61467 ... 307508 307509 307510]
[    0     1     2 ... 61814 61816 61828]
[     0      1      2 ... 307508 307509 307510]
[ 61465  61466  61467 ... 123093 123094 123095]
[     0      1      2 ... 307508 307509 307510]
[121993 122029 122030 ... 184624 184625 184626]
[     0      1      2 ... 307508 307509 307510]
[183271 183273 183278 ... 246095 246097 246099]
[     0      1      2 ... 246095 246097 246099]
[244997 245014 245015 ... 307508 307509 307510]


# 【問題2】グリッドサーチ
* グリッドサーチをパイプラインの中に組み込む
* ついでにパイプラインを修正、クロスバリデーションに対応した形にしたい

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

def c_pipeline(data, clf_dict):
    accuracy_dict = {}
    confusion_matrix_dict = {}
    
    for i, clf_key in enumerate(clf_dict):
        # データ分割
        X_train, X_test, y_train, y_test = train_test_split(data[i]['X'], data[i]['y'])
        
        # 学習
        clf_dict[clf_key].fit(X_train, y_train) 

        # 予測実施
        y_pred = clf_dict[clf_key].predict(X_test)

        
        accuracy_dict[clf_key] = metrics.accuracy_score(y_test, y_pred)
        
        confusion_matrix_dict[clf_key] = metrics.confusion_matrix(y_test, y_pred)
    
    return accuracy_dict, confusion_matrix_dict